<a href="https://www.kaggle.com/code/carolinariddick/fmri-analysis-using-glm-and-dmn-visualization?scriptVersionId=271517430" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# fMRI Analysis Using GLM and DMN Visualization

In this results simulation, we will analyse a dataset from a “listen vs. rest” task. Using this methodology, the researchers aimed to study the brain areas involved in sound processing.

The task is very simple: compare brain responses when people were listening to something versus when they were not.

Objectives of this notebook:

- Learn the basics of the nilearn library for analysing fMRI data.

- Learn how to load and visualize fMRI data in Python.

- Understand how to perform a statistical analysis on this type of signals.

- Learn how to interpret the results.

- Learn how to write a brief report summarizing the findings.

### Step 1: Install dependencies

In [ ]:
!pip install --quiet --upgrade \
    nilearn==0.10.2 \
    matplotlib==3.7.2 \
    numpy==1.25.2 \
    pandas==2.1.1 \
    scipy==1.11.2 \
    scikit-learn==1.5.2 \
    nibabel==5.0.2 \
    joblib==1.3.2

### Clean the cache

In [ ]:
!pip check --quiet

### Step 2: Import dependencies

In [ ]:
import zipfile
import os
from nilearn.plotting import plot_anat, plot_img, plot_stat_map
from nilearn.image import concat_imgs, mean_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_contrast_matrix
from nilearn.glm import threshold_stats_img
from nilearn.image import load_img
from nilearn.datasets import fetch_spm_auditory
import pandas as pd
import numpy as np
import matplotlib as plt
import os
from collections import namedtuple
import nibabel as nib
from nilearn.image import index_img
from nilearn.plotting import plot_stat_map
import matplotlib.pyplot as plt
from nilearn.reporting import get_clusters_table
from nilearn import datasets, plotting

### Step 3: Import dataset 

#### We will use the dataset I have downloaded in Kaggle.

In Nilearn, we can use datasets that are already preprocessed and ready for analysis.
For this task, we are going to use the fetch_spm_auditory dataset.

To import it, we can do the following:

In [ ]:
subject_data = fetch_spm_auditory() 

print(*subject_data.func[:5], sep="\n") 

In [ ]:
# Check the dataset attributes
print("Dataset attributes:")
print(subject_data.keys()) 

# Display the first 5 functional image files
print("\nFirst 5 functional image files:")
print(subject_data.func[:5])

# Inspect the type and dimensions of the first file
first_func_file = subject_data.func[0]
print("\nType of the first functional file:", type(first_func_file))

# Load it with nibabel to inspect the image
img = nib.load(first_func_file)
print("Nibabel object type:", type(img))
print("Image shape (x, y, z, t):", img.shape)
print("Number of dimensions:", img.ndim)

# Display values
data = img.get_fdata()
print("Data type in the array:", data.dtype)
print("Min/Max values in the image:", data.min(), data.max())


In [ ]:
# How many 3D volumes has my 4D file?
img = nib.load(first_func_file)
print("Shape of the 4D file:", img.shape)
print("Number of 3D volumes:", img.shape[-1])

In [ ]:
# List of all functional files for the subject
func_files = subject_data.func  # ['fM00223_004.img', 'fM00223_005.img', ...]

# Load each file as a Nifti1Image
imgs_3d = [nib.load(f) for f in func_files]

In [ ]:
img_4d = concat_imgs(imgs_3d)
print("Shape of the 4D merged file:", img_4d.shape)  # (64,64,64,n_vols)

In [ ]:
# Showing the first 3 volumes
for i in range(3):
    vol_3d = index_img(img_4d, i)
    plot_stat_map(
        vol_3d,
        title=f"3D Volume {i+1}",
        cmap='viridis',
        colorbar=True
    )

plt.show()

In [ ]:
subject_data.anat

In [ ]:
anat_img = subject_data.anat
plot_anat(anat_img, title="Anatomic Image")
plt.show()

### Step 4 : Visualize Data

EDA (Exploratory Data Analysis) is about understanding the nature of your data.
A quick way to do this is by using Pandas functions such as describe, info, shape, etc.
However, this only applies when the data is structured — in other words, when it is organised in tables, like in Excel files.

Another way to understand your data is by visualising it.
Here, we are going to use nilearn.plotting to visualise the subject’s data:

#### Exercise 3. Visualise the brain using plot_anat and plot_img functions

### Step 5. Concatenate images

We need to do this to concatenate all the 3D images into a single 4D image. After that, we calculate an average to create a background image, which we will later use to visualise the activations in different regions of the brain.

#### Exercise 4. Compute the mean of the fMRI image

For this, you should use the mean_img function from nilearn.image (already imported).

In [ ]:
# concat all the images 4D
fmri_img = concat_imgs(subject_data.func)
mean_img = mean_img(fmri_img)

print('Mean IMG:',mean_img)

In [ ]:
plot_img(mean_img, title="Mean Brain (Mean fMRI)")
plt.show()

### Step 6: Create a table for the experiment events

Now we need to provide a description of the experiment — that is, to define the timing of the auditory stimulation and the rest periods. This is usually provided in an events.tsv file. Fortunately, the path to this file is already included in the dataset we are working with.

In [ ]:
events = pd.read_table(subject_data["events"])
# events.shape
events

In [ ]:
# Create the GLM model
fmri_glm = FirstLevelModel(t_r=7, hrf_model='spm')

fmri_glm = fmri_glm.fit(fmri_img, events=events)

# Get the design matrix 
design_matrix = fmri_glm.design_matrices_[0]

print("Design matrix shape:", design_matrix.shape)
print(design_matrix.head())

# Plot the design matrix 
plot_design_matrix(design_matrix)


### Step 7. Detect voxels with significant effects

To access the estimated coefficients (the GLM model betas), we create a contrast with a single ‘1’ in each of the relevant columns.

The purpose of the contrast is to select specific columns of the model in order to study the associated statistics.

Here, we can define canonical contrasts that consider each effect in isolation — let’s call them “conditions” — and then define a contrast that captures the difference between these conditions.

In [ ]:
conditions = {"active": np.zeros(16), "rest": np.zeros(16)}
conditions["active"][0] = 1
conditions["rest"][1] = 1

Now we can compare the two conditions in the experiment (active vs. rest) by defining the corresponding contrast:

#### Exercise 5: Create a variable called "active_minus_rest" that computes the difference between the "active" and "rest" conditions

In [ ]:
active_minus_rest = conditions["active"] - conditions["rest"]

#### Exercise 6. Visualise the conditions in a Contrast Matrix

In [ ]:
fmri_glm = FirstLevelModel(
    t_r=7,
    noise_model="ar1",
    standardize=False,
    hrf_model="spm",
    drift_model="cosine",
    high_pass=0.01,
)

fmri_glm = fmri_glm.fit(fmri_img, events)

design_matrix = fmri_glm.design_matrices_[0]

We can use the function plot_contrast_matrix(data, design_matrix=design_matrix) to visualise:

In [ ]:
print(design_matrix.columns)
print(design_matrix.shape)

In [ ]:
plot_contrast_matrix(active_minus_rest, design_matrix=design_matrix)

Next, we need to compute the “estimated effect.”

This is measured in units of BOLD signal, but it doesn’t provide strong statistical guarantees, since it doesn’t account for the associated variance. In the following cells, you will see how this is done.

In [ ]:
eff_map = fmri_glm.compute_contrast(
    active_minus_rest, output_type="effect_size"
)

To obtain statistical significance, we first need to create a t-statistic, which we will then convert to a z-score.

The z-score scale means that values are standardized to match a standard Gaussian distribution (mean = 0, variance = 1) across all voxels.

In [ ]:
z_map = fmri_glm.compute_contrast(active_minus_rest, output_type="z_score")

#### Step 8. Visualise the Significant Results

⚠️ Important: remember that this dataset specifically focuses on the auditory cortex, not the DMN. I’ve created a plot below that you can use in your work, since the significant results will only indicate activation in the auditory cortex.

### 📚 Some Notes
#### Interpreting the Statistical Maps

Once we have generated the statistical maps, it is important to understand how to interpret the visualized results. In this case, we are working with Z-maps, where Z-values indicate how far the observed effect is from the mean, in terms of standard deviations.

### What does a Z-value mean?

Positive and significant Z-values: A Z-value greater than 3 (Z > 3) indicates significantly higher activation in a specific brain region compared to the reference condition (here, the rest state). For example, high Z-values in the auditory cortex suggest significant activation during the auditory task.

Negative Z-values: Negative Z-values indicate deactivation in that region during the task compared to rest. Deactivated areas often include regions of the Default Mode Network (DMN), such as the Precuneus or Medial Prefrontal Cortex, which typically show higher activity during rest.

### Statistical significance

Z-values are scaled to a standard Gaussian distribution, allowing us to identify brain regions with significant activation. We typically apply a statistical threshold (e.g., Z > 3) to visualize only the regions showing significant activation. However, note that this threshold is arbitrary and does not account for the false positive rate (Type I errors).

To correct for this, stricter statistical corrections can be applied, such as False Discovery Rate (FDR) or Family-Wise Error Rate (FPR). For instance, an FDR correction with α = 0.05 reduces false positives and ensures the observed activations are more robust and reliable.

### Activation Clusters

Finally, small activation clusters that could be artifacts or noise are often removed. In this analysis, a 10-voxel threshold was applied, ensuring that only sufficiently large and significant clusters are considered. This helps identify the most relevant brain regions in terms of activation.

#### Exercise 7. Add Parameters to the plot_stat_map() Function

For this example, we will arbitrarily use:

1. a threshold of 3.0 in Z-score.

2. `display_mode='z'`

3. `cut_coords=3`

In [ ]:
plot_stat_map(
    z_map,
    bg_img=mean_img,
    threshold= 3.0,
    display_mode= "z",
    cut_coords= 3,
    black_bg=True,
    title= "Active minus Rest (Z>3)",
)
plt.show()

In [ ]:
plot_stat_map(
    z_map,
    bg_img=mean_img,
    threshold= 3.0,
    display_mode= "z",
    cut_coords= 3,
    black_bg=True,
    title= "Active minus Rest (Z>3)",
)
plt.show()

Here we use an arbitrary threshold of 3.0, but the threshold should also take into account the risk of false detections (also known as Type I errors).

One suggestion is to control the false positive rate (FPR, denoted by alpha) at a certain level, for example 0.001. This means there is a 0.1% chance of declaring an inactive voxel as active.

We can do this as follows:

In [ ]:
_, threshold = threshold_stats_img(z_map, alpha=0.001, height_control="fpr")
print(f"Uncorrected p<0.001 threshold: {threshold:.3f}")
plot_stat_map(
    z_map,
    bg_img=mean_img,
    threshold=threshold,
    display_mode="z",
    cut_coords=3,
    black_bg=True,
    title="Active minus Rest (p<0.001)",
)
plt.show()

The problem is that with this approach, we expect 0.001 × n_voxels false positives, even when they are inactive (which can be tens to hundreds of voxels).

A popular alternative is to control the expected proportion of false discoveries among the detections, known as the false discovery rate (FDR).

We can do it as follows:

In [ ]:
_, threshold = threshold_stats_img(z_map, alpha=0.05, height_control="fdr")
print(f"False Discovery rate = 0.05 threshold: {threshold:.3f}")
plot_stat_map(
    z_map,
    bg_img=mean_img,
    threshold=threshold,
    display_mode="z",
    cut_coords=3,
    black_bg=True,
    title="Active minus Rest (fdr=0.05)",
)
plt.show()

Finally, a good practice is to discard isolated voxels (also known as "small clusters") from these images.

It is possible to generate a thresholded map in which small clusters are removed by providing a cluster_threshold argument. In this case, clusters smaller than 10 voxels will be discarded.

In [ ]:
clean_map, threshold = threshold_stats_img(
    z_map, alpha=0.05, height_control="fdr", cluster_threshold=10
)
plot_stat_map(
    clean_map,
    bg_img=mean_img,
    threshold=threshold,
    display_mode="z",
    cut_coords=3,
    black_bg=True,
    title="Active minus Rest (fdr=0.05), clusters > 10 voxels",
)
plt.show()

Now we have a statistical analysis of the regions that are significantly more active compared to others.

In the image, we can see that there is very significant activity in the auditory cortex. This makes perfect sense, as the experimental task was to listen to a sound versus resting (doing nothing).

#### Step 9: Save the results in a Table

To save the coordinates of significant results, it’s good practice to have them organized and structured in a table.

Fortunately, we can do this very easily using the `get_clusters_table` function.

In [ ]:
table = get_clusters_table(
    z_map, stat_threshold=threshold, cluster_threshold=20
)

table

### Step 10: Write down the results

#### Example 1

In our brain activation analysis for the listening versus rest task, we used a General Linear Model (GLM) to study differences in activation within the Default Mode Network (DMN). Through contrast analysis (listening vs. resting), a Z-map was generated showing significant brain activation for the auditory task.

The results indicated significantly higher activations in regions associated with the DMN when participants were at rest compared to the auditory task (Z > 3, p < 0.001 uncorrected). After applying a False Discovery Rate (FDR) correction with a threshold of 0.05, we observed that the most involved areas were concentrated in …

Finally, by removing small clusters of fewer than 10 voxels, we observed that the most significant activations remained in key areas of the DMN, suggesting a robust effect in these regions. The results are presented in Table 1, summarizing the coordinates of voxels with significant activation.

Figure 1. Z-map of the differences between the auditory task and rest, showing significant activation in the DMN (Z > 3, p < 0.001).

#### Example 2

In this analysis, we implemented a General Linear Model (GLM) to compare brain activation during an auditory task versus a resting state, aiming to study the dynamics of the Default Mode Network (DMN). The data were analyzed using a block design with a TR of 7 seconds, and the canonical Hemodynamic Response Function (HRF) from SPM was applied to model the BOLD responses.
The results of the contrast between listening and resting conditions produced a Z-map reflecting significant activation across multiple brain regions. When defining the contrast “listen minus rest,” robust activation was observed in bilateral auditory cortex areas during the auditory task. In contrast, relative deactivation was identified in regions belonging to the DMN, particularly the Precuneus, Medial Prefrontal Cortex (MPFC), and Inferior Parietal Cortex.
An estimated effects map was generated for the “listen vs. rest” contrast, showing the magnitude of effects in BOLD signal units. However, these initial results did not provide full statistical validation, as the associated variance was not considered. To address this, a Z-map was calculated, allowing estimation of statistical significance based on variance. With a threshold of Z > 3, significant activations were detected in primary auditory cortex, while deactivations were observed in DMN areas.
To improve the reliability of the results and reduce the risk of false positives, two statistical corrections were applied. The control of the False Positive Rate (FPR) set a threshold of p < 0.001, ensuring that fewer than 0.1% of inactive voxels were incorrectly classified as active. Additionally, a False Discovery Rate (FDR) correction with α = 0.05 was implemented, focusing results on significant regions, particularly in the auditory cortex and DMN. Finally, a cluster threshold of 10 voxels removed small spurious clusters, consolidating the results in key brain regions.

In [ ]:
# Create and fit the GLM
fmri_glm = FirstLevelModel(t_r=7, hrf_model='spm')
fmri_glm = fmri_glm.fit(fmri_img, events=events)

# Check the design matrix column names
design_matrix = fmri_glm.design_matrices_[0]
print("Design matrix columns:", design_matrix.columns)

# Create the contrast 'active minus rest'
contrast_vector = np.array([1, -1, 0])

# Compute the contrast as a Z-score map
z_map = fmri_glm.compute_contrast(contrast_vector, output_type='z_score')

plot_stat_map(z_map, threshold=3.0, display_mode='z', cut_coords=3, title='Active minus Rest')

### Step 11: Work figures

In [ ]:
import numpy as np
import nibabel as nib
from nilearn import plotting, datasets
from nilearn.image import threshold_img

# Step 1: Define prefrontal cortex coordinates and peak Z-statistics
coordinates_and_peaks = [
    (-60, -6, 42, 9.81),  # MNI coordinates of prefrontal cortex with peak Z-stat values
    (-63, 6, 36, 8.60),
    (-63, 0, 42, 8.43),
    (-48, -15, 39, 8.36),
    (45, -12, 42, 7.59)
]

# Step 2: Create an empty brain image (MNI space: 91x109x91) with a basic affine
brain_shape = (91, 109, 91)
affine = np.array([[2., 0., 0., -90.],
                   [0., 2., 0., -126.],
                   [0., 0., 2., -72.],
                   [0., 0., 0., 1.]])
brain_data = np.zeros(brain_shape)

# Step 3: Place Z-statistic values into the brain image at the MNI coordinates
for x, y, z, z_value in coordinates_and_peaks:
    voxel_coord = np.round(np.linalg.inv(affine).dot([x, y, z, 1]))[:3].astype(int)
    # Check bounds to avoid indexing errors
    if all(0 <= voxel_coord[i] < brain_shape[i] for i in range(3)):
        brain_data[tuple(voxel_coord)] = z_value

# Step 4: Create a Nifti image for the simulated activations
simulated_img = nib.Nifti1Image(brain_data, affine)

# Step 5: Apply a simple statistical threshold (uncorrected p<0.001 ~ Z>3.09)
threshold = 3.09
thresholded_img = threshold_img(simulated_img, threshold=threshold)

# Step 6: Load the MNI template as the background image
mean_img = datasets.load_mni152_template()

# Step 7: Visualize the statistical map with thresholding
plotting.plot_stat_map(
    thresholded_img,
    bg_img=mean_img,      # Use MNI template as background
    threshold=threshold,  # Apply the threshold
    display_mode="z",
    cut_coords=(0, 50, 10),  # Focus on prefrontal cortex
    black_bg=True,        # Black background for clarity
    title="Simulated Prefrontal Cortex Activation"
)

plotting.show()


In [ ]:
# Step 1: Define the prefrontal cortex coordinates and peak activations
coordinates_and_peaks = [
    (-60, -6, 42, 9.81),  # Example prefrontal cortex MNI coordinates
    (-63, 6, 36, 8.60),
    (-63, 0, 42, 8.43),
    (-48, -15, 39, 8.36),
    (45, -12, 42, 7.59)  # You can replace these with the prefrontal activations from your table
]

# Step 2: Create an empty brain image (MNI space: 91x109x91) with a basic affine
brain_shape = (91, 109, 91)
affine = np.array([[2., 0., 0., -90.],
                   [0., 2., 0., -126.],
                   [0., 0., 2., -72.],
                   [0., 0., 0., 1.]])
brain_data = np.zeros(brain_shape)

# Step 3: Place peak activations into the brain image at the MNI coordinates
for x, y, z, peak in coordinates_and_peaks:
    voxel_coord = np.round(np.linalg.inv(affine).dot([x, y, z, 1]))[:3].astype(int)
    brain_data[tuple(voxel_coord)] = peak

# Step 4: Create a Nifti image for the simulated activations
simulated_img = nib.Nifti1Image(brain_data, affine)

plotting.plot_stat_map(simulated_img, display_mode='ortho', threshold=3.0,
                       title="", cut_coords=(0, 50, 10))

plotting.plot_glass_brain(simulated_img, threshold=3.0, display_mode='lzry', colorbar=True,
                          title="")

plotting.show()

### DMN Graphs

In this project, it was decided to use the DiFuMo atlas (or Harvard–Oxford, depending on the case) instead of the original MSDL atlas. The main reason is that the MSDL atlas, although widely used for identifying networks such as the Default Mode Network (DMN), is no longer available on its official servers, and there are no reliable public mirrors for downloading it.
Attempts to use fetch_atlas_msdl() resulted in connection errors due to network blocks and the fact that the original files have been removed.
For this reason, an alternative functional atlas (DiFuMo) was chosen because it:


Is available for download from accessible and reliable servers (OSF),


Is also functional and probabilistic, allowing identification of brain networks similarly to MSDL,


Ensures reproducibility and compatibility with environments such as Kaggle.


This choice guarantees that analyses and visualizations can be performed without download errors or incompatibilities, while maintaining the functional focus originally intended.

In [ ]:
# # Downloas the MSDL atlas
# msdl_atlas = datasets.fetch_atlas_msdl()

# # This is the link
# https://team.inria.fr/parietal/files/2015/01/MSDL_rois.zip

In [ ]:

# # Extract the corresponding nodes to the Default Mode Network (DMN)
# dmn_nodes = image.index_img(msdl_atlas.maps, [3, 4, 5, 6])

# # Visualize the nodes from DMN
# plotting.plot_prob_atlas(dmn_nodes, cut_coords=(0, -60, 29), draw_cross=False,
#                          annotate=False, title="DMN nodes in MSDL atlas")


In [ ]:
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
plotting.plot_prob_atlas(atlas.maps, title="Harvard-Oxford Atlas")

In [ ]:
from nilearn import image, datasets, plotting

# Get the DiFuMo dataset
difumo = datasets.fetch_atlas_difumo()

# difumo.maps is a Nifti image with all the regions
# We choose some indexes as example
dmn_nodes = image.index_img(difumo.maps, [0, 1, 2, 3])

plotting.plot_prob_atlas(
    dmn_nodes, cut_coords=(0, -60, 29),
    draw_cross=False, annotate=False,
    title="DMN nodes (DiFuMo atlas)"
)

In [ ]:
# Extract the nodes corresponding to the Default Mode Network (DMN)
dmn_nodes = image.index_img(difumo.maps, [3, 4, 5, 6])

# Visualize the DMN nodes
plotting.plot_prob_atlas(
    dmn_nodes, 
    cut_coords=(0, -60, 29), 
    draw_cross=False,
    annotate=False, 
    title="DMN nodes in MSDL atlas"
)

| Atlas          | Type       | Functionality                             | Safe to download on Kaggle? |
| -------------- | ---------- | ----------------------------------------- | --------------------------- |
| **DiFuMo**     | Functional | Functional networks, probabilistic        | Yes                         |
| **Schaefer**   | Functional | Functional networks, multiple resolutions | Yes                         |
| Harvard–Oxford | Anatomical | Structural regions                        | Yes                         |
